# 3D map simulation

## Prerequisites

- Knowledge of 3D extraction and datasets used in gammapy, see for instance the [first analysis tutorial](./3D_analysis.ipynb)

## Context

To simulate a specific observation, it is not always necessary to simulate the full photon list. For many uses cases, simulating directly a reduced binned dataset is enough: the IRFs reduced in the correct geometry are combined with a source model to predict an actual number of counts per bin. The latter is then used to simulate a reduced dataset using Poisson probability distribution.

This can be done to check the feasibility of a measurement (performance / sensitivity study), to test whether fitted parameters really provide a good fit to the data etc.

Here we will see how to perform a 3D simulation of a CTA observation, assuming both the spectral and spatial morphology of an observed source.

**Objective: simulate a 3D observation of a source with CTA using the CTA 1DC response and fit it with the assumed source model.**

## Proposed approach:

Here we can't use the regular observation objects that are connected to a `DataStore`. Instead we will create a fake `~gammapy.data.Observation` that contain some pointing information and the CTA 1DC IRFs (that are loaded with `~gammapy.irf.load_cta_irfs`).

Then we will create a `~gammapy.datasets.MapDataset` geometry and create it with the `~gammapy.makers.MapDatasetMaker`.

Then we will be able to define a model consisting of  a `~gammapy.modeling.models.PowerLawSpectralModel` and a `~gammapy.modeling.models.GaussianSpatialModel`. We will assign it to the dataset and fake the count data.


## Imports and versions

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.irf import load_irf_dict_from_file
from gammapy.maps import WcsGeom, MapAxis
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    GaussianSpatialModel,
    SkyModel,
    Models,
    FoVBackgroundModel,
)
from gammapy.makers import MapDatasetMaker, SafeMaskMaker
from gammapy.modeling import Fit
from gammapy.data import Observation, FixedPointingInfo, PointingMode, observatory_locations
from gammapy.datasets import MapDataset
from gammapy.estimators import FluxPointsEstimator
from scipy.stats import chi2
from gammapy.stats.utils import ts_to_sigma
from IPython.core.display import HTML 

In [3]:
!gammapy info --no-envvar --no-dependencies --no-system

 tcsh initialization done
/home/khelifi/MesProgrammes/gammapy//gammapy: Permission non accordée.


## Simulation

We will simulate using the CTA-1DC IRFs shipped with gammapy. Note that for dedictaed CTA simulations, you can simply use [`Observation.from_caldb()`]() without having to externally load the IRFs

In [4]:
# Loading IRFs
irfs = load_irf_dict_from_file("$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits")

Invalid unit found in background table! Assuming (s-1 MeV-1 sr-1)


In [5]:
# Define the observation parameters (typically the observation duration and the pointing position):
livetime = 10.0 * u.hr
pointing_position = SkyCoord(0, 0, unit="deg", frame="galactic")
pointing = FixedPointingInfo(
    mode=PointingMode.POINTING,
    fixed_icrs=pointing_position.icrs,
)

In [6]:
# Define map geometry for binned simulation
energy_reco = MapAxis.from_edges(
    np.logspace(-1.0, 2.0, 15), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(6, 6),
    frame="galactic",
    axes=[energy_reco],
)
# It is usually useful to have a separate binning for the true energy axis
energy_true = MapAxis.from_edges(
    np.logspace(-1.5, 2.5, 30), unit="TeV", name="energy_true", interp="log"
)

empty = MapDataset.create(
    geom, name="dataset-simu", energy_axis_true=energy_true
)

In [7]:
# Define sky model to used simulate the data.
# Here we use a Gaussian spatial model and a Power Law spectral model.
spatial_model = GaussianSpatialModel(
    lon_0="0.2 deg", lat_0="0.1 deg", sigma="0.3 deg", frame="galactic"
)
spectral_model = PowerLawSpectralModel(
    index=2.2, amplitude="1e-12 cm-2 s-1 TeV-1", reference="1 TeV"
)
model_simu = SkyModel(
    spatial_model=spatial_model,
    spectral_model=spectral_model,
    name="model-simu",
)

bkg_model = FoVBackgroundModel(dataset_name="dataset-simu")

models = Models([model_simu, bkg_model])
print(models)

Models

Component 0: SkyModel

  Name                      : model-simu
  Datasets names            : None
  Spectral model type       : PowerLawSpectralModel
  Spatial  model type       : GaussianSpatialModel
  Temporal model type       : 
  Parameters:
    index                         :      2.200   +/-    0.00             
    amplitude                     :   1.00e-12   +/- 0.0e+00 1 / (TeV s cm2)
    reference             (frozen):      1.000       TeV         
    lon_0                         :      0.200   +/-    0.00 deg         
    lat_0                         :      0.100   +/-    0.00 deg         
    sigma                         :      0.300   +/-    0.00 deg         
    e                     (frozen):      0.000                   
    phi                   (frozen):      0.000       deg         

Component 1: FoVBackgroundModel

  Name                      : dataset-simu-bkg
  Datasets names            : ['dataset-simu']
  Spectral model type       : PowerLawNormSpec

Now, comes the main part of dataset simulation. We create an in-memory observation and an empty dataset. We then predict the number of counts for the given model, and Poission fluctuate it using `fake()` to make a simulated counts maps. Keep in mind that it is important to specify the `selection` of the maps that you want to produce 

In [8]:
# Create an in-memory observation
obs = Observation.create(pointing=pointing, livetime=livetime, irfs=irfs)
print(obs)

Observation

	obs id            : 0 
 	tstart            : 51544.00
	tstop             : 51544.42
	duration          : 36000.00 s
	pointing (icrs)   : 266.4 deg, -28.9 deg

	deadtime fraction : 0.0%



In [9]:
# Make the MapDataset
maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])

maker_safe_mask = SafeMaskMaker(methods=["offset-max"], offset_max=4.0 * u.deg)

dataset = maker.run(empty, obs)
dataset = maker_safe_mask.run(dataset, obs)

In [10]:
%%time

# Add the model on the dataset and Poission fluctuate
dataset.models = models
dataset.fake()
# Do a print on the dataset - there is now a counts maps
print(dataset)

MapDataset
----------

  Name                            : dataset-simu 

  Total counts                    : 812421 
  Total background counts         : 810346.54
  Total excess counts             : 2074.46

  Predicted counts                : 812153.07
  Predicted background counts     : 810346.54
  Predicted excess counts         : 1806.52

  Exposure min                    : 2.24e+04 m2 s
  Exposure max                    : 2.06e+11 m2 s

  Number of total bins            : 1260000 
  Number of fit bins              : 1251432 

  Fit statistic type              : cash
  Fit statistic value (-2 log(L)) : 290686.92

  Number of models                : 2 
  Number of parameters            : 11
  Number of free parameters       : 6

  Component 0: SkyModel
  
    Name                      : model-simu
    Datasets names            : None
    Spectral model type       : PowerLawSpectralModel
    Spatial  model type       : GaussianSpatialModel
    Temporal model type       : 
    Parame

In [11]:
dataset.counts.geom

WcsGeom

	axes       : ['lon', 'lat', 'energy']
	shape      : (300, 300, 14)
	ndim       : 3
	frame      : galactic
	projection : CAR
	center     : 0.0 deg, 0.0 deg
	width      : 6.0 deg x 6.0 deg
	wcs ref    : 0.0 deg, 0.0 deg

Now use this dataset as you would in all standard analysis. You can plot the maps, or proceed with your custom analysis. 
In the next section, we show the standard 3D fitting as in [analysis_3d](analysis_3d.ipynb).

In [12]:
# To plot, eg, counts:
dataset.counts.smooth(0.2 * u.deg).plot_interactive(
    add_cbar=True, stretch="linear"
)

interactive(children=(SelectionSlider(continuous_update=False, description='Select energy:', layout=Layout(wid…

# Exercises

## 1. Retrieve the SkyModel parameters with a 3D analysis fit from the simulated dataset

For this exercise, you will do a usual 3D fit with the same model used to simulated the data and see the stability of the simulations. And estimate the signifiance of the source.

Objectives: Often, it is useful to simulate many such datasets and look at the distribution of the reconstructed parameters.

In [ ]:
models_fit = models.copy()
XXX ....

And look at the results of your neighbour and interpret the comparisons!

## 2. Using simulations, determine the minimal time to detect significantly an exponential cut-off in the spectrum of a weak extended source
Estimating the significance of an Exponential Cutoff in the spectrum for each simulation, and change the observation time for each simulation.
The normalisation at 1TeV of the source can be set at 1e-12 cm-2 s-1 TeV-1, the index at 2.2 and the cut-off energy at 40 TeV. The Gaussian sigma is set to 0.3 deg.

Spectral models can be found here: https://docs.gammapy.org/1.1/user-guide/model-gallery/index.html

In [ ]:
#Generate a new dataset with an Exponential Cutoff power-law with a E_cut = 50 TeV
#==> redefine a sky model and change the spectral model with an Exponential Cutoff power-law 
#Fit the dataset with a skymodel assuming an Exponential Cutoff power-law and save the likelihood value
#Fit the dataset with a skymodel assuming a power-law and save the likelihood value

#If you want to produce a histogram of reconstructed cut-off energy, you can run this analysis multiple times